In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.pipeline import Pipeline
import sklearn.preprocessing as preprocessing
from IPython.core.interactiveshell import InteractiveShell

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest, SelectFromModel
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.metrics import classification_report, roc_auc_score, ConfusionMatrixDisplay

InteractiveShell.ast_node_interactivity = 'all'

to_show = True
to_save = False
is_sample = False
is_sample = True
limit_rows = 1000

In [2]:
# This data cleaning part gets ideas from https://www.kaggle.com/apryor6/detailed-cleaning-visualization-python
# HAN YIKAI takes charge of feature 1-9
# official file
train_file = '../input/train_ver2.csv'
test_file = '../input/test_ver2.csv'

out_path = '../output/'


def save_df2file(data_frame: pd.DataFrame, name: str):
    data_frame.to_csv(f'{out_path}{name}.csv')

# sns.set(rc = {'figure.figsize':(20,18)})


In [3]:
# test_df = pd.read_csv(test_file,
#                       dtype={'sexo': str,
#                              'ind_nuevo': str,
#                              'ult_fec_cli_1t': str,
#                              'indext': str},
#                       # nrows=limit_rows,
#                       low_memory=False
#                       )

In [4]:
df = pd.read_csv(train_file,
                 dtype={'sexo': str,
                        'ind_nuevo': str,
                        'ult_fec_cli_1t': str,
                        'indext': str},
                 # nrows=limit_rows,
                 low_memory=False
                 )

# resample the data for saving memory and shuffle the dataset
if is_sample:
    df = df.sample(frac=0.01)

y_name_list = list(df.iloc[:, 24:].columns)
x_name_list = list(df.iloc[:, :24].columns)
# df = df[order]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136473 entries, 2428958 to 943302
Data columns (total 48 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   fecha_dato             136473 non-null  object 
 1   ncodpers               136473 non-null  int64  
 2   ind_empleado           136213 non-null  object 
 3   pais_residencia        136213 non-null  object 
 4   sexo                   136211 non-null  object 
 5   age                    136473 non-null  object 
 6   fecha_alta             136213 non-null  object 
 7   ind_nuevo              136213 non-null  object 
 8   antiguedad             136473 non-null  object 
 9   indrel                 136213 non-null  float64
 10  ult_fec_cli_1t         230 non-null     object 
 11  indrel_1mes            134963 non-null  object 
 12  tiprel_1mes            134963 non-null  object 
 13  indresi                136213 non-null  object 
 14  indext                 136213 

In [5]:
df.loc[:, y_name_list].isna().sum()

ind_ahor_fin_ult1      0
ind_aval_fin_ult1      0
ind_cco_fin_ult1       0
ind_cder_fin_ult1      0
ind_cno_fin_ult1       0
ind_ctju_fin_ult1      0
ind_ctma_fin_ult1      0
ind_ctop_fin_ult1      0
ind_ctpp_fin_ult1      0
ind_deco_fin_ult1      0
ind_deme_fin_ult1      0
ind_dela_fin_ult1      0
ind_ecue_fin_ult1      0
ind_fond_fin_ult1      0
ind_hip_fin_ult1       0
ind_plan_fin_ult1      0
ind_pres_fin_ult1      0
ind_reca_fin_ult1      0
ind_tjcr_fin_ult1      0
ind_valo_fin_ult1      0
ind_viv_fin_ult1       0
ind_nomina_ult1      155
ind_nom_pens_ult1    155
ind_recibo_ult1        0
dtype: int64

In [6]:
y_name_list

['ind_ahor_fin_ult1',
 'ind_aval_fin_ult1',
 'ind_cco_fin_ult1',
 'ind_cder_fin_ult1',
 'ind_cno_fin_ult1',
 'ind_ctju_fin_ult1',
 'ind_ctma_fin_ult1',
 'ind_ctop_fin_ult1',
 'ind_ctpp_fin_ult1',
 'ind_deco_fin_ult1',
 'ind_deme_fin_ult1',
 'ind_dela_fin_ult1',
 'ind_ecue_fin_ult1',
 'ind_fond_fin_ult1',
 'ind_hip_fin_ult1',
 'ind_plan_fin_ult1',
 'ind_pres_fin_ult1',
 'ind_reca_fin_ult1',
 'ind_tjcr_fin_ult1',
 'ind_valo_fin_ult1',
 'ind_viv_fin_ult1',
 'ind_nomina_ult1',
 'ind_nom_pens_ult1',
 'ind_recibo_ult1']

In [7]:
# 评估
def print_evaluation(y_true, y_pred):
    pass


In [8]:
# 复用函数

# 统计缺失值的列
def cal_loss(data):
    fil = data.iloc[:, :24].isnull().any()
    loss_column = fil[fil == True].index
    res = data.loc[:, loss_column].isnull().sum()
    print(res)

#
# def renew_df(base_df: pd.DataFrame):
#     new_df = base_df
#     base_df = df.copy(deep=True)
#     return new_df

In [9]:
# 删除y值空缺的行
df.drop(df[df['ind_nomina_ult1'].isnull()].index, inplace=True)

# 调整label的格式节省空间, 能节约2.1G 空间, 将近一半
df.loc[:, y_name_list] = df.loc[:, y_name_list].astype('uint8')
df.loc[:, y_name_list].value_counts(dropna=False)

/tmp/ipykernel_71272/3066333033.py:5: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, y_name_list] = df.loc[:, y_name_list].astype('uint8')


ind_ahor_fin_ult1  ind_aval_fin_ult1  ind_cco_fin_ult1  ind_cder_fin_ult1  ind_cno_fin_ult1  ind_ctju_fin_ult1  ind_ctma_fin_ult1  ind_ctop_fin_ult1  ind_ctpp_fin_ult1  ind_deco_fin_ult1  ind_deme_fin_ult1  ind_dela_fin_ult1  ind_ecue_fin_ult1  ind_fond_fin_ult1  ind_hip_fin_ult1  ind_plan_fin_ult1  ind_pres_fin_ult1  ind_reca_fin_ult1  ind_tjcr_fin_ult1  ind_valo_fin_ult1  ind_viv_fin_ult1  ind_nomina_ult1  ind_nom_pens_ult1  ind_recibo_ult1
0                  0                  1                 0                  0                 0                  0                  0                  0                  0                  0                  0                  0                  0                  0                 0                  0                  0                  0                  0                  0                 0                0                  0                  60791
                                      0                 0                  0                 0   

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136318 entries, 2428958 to 943302
Data columns (total 48 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   fecha_dato             136318 non-null  object 
 1   ncodpers               136318 non-null  int64  
 2   ind_empleado           136213 non-null  object 
 3   pais_residencia        136213 non-null  object 
 4   sexo                   136211 non-null  object 
 5   age                    136318 non-null  object 
 6   fecha_alta             136213 non-null  object 
 7   ind_nuevo              136213 non-null  object 
 8   antiguedad             136318 non-null  object 
 9   indrel                 136213 non-null  float64
 10  ult_fec_cli_1t         230 non-null     object 
 11  indrel_1mes            134963 non-null  object 
 12  tiprel_1mes            134963 non-null  object 
 13  indresi                136213 non-null  object 
 14  indext                 136213 

In [11]:
# fecha_dato 日期
# ncodpers 客户代码
# ind_empleado 员工
# pais_residencia 国藉
# sexo 性别,
# age 年龄
# fecha_alta 注册日期,
# ind_nuevo 新的,
# antiguedad 古代,
# indrel 最后访问日期, 一个月indrel
# ult_fec_cli_1t  作为主要客户的日期
# indrel_1mes  1 (First/Primary customer), 2 (co-owner ),P (Potential),3 (former primary), 4(former co-owner)
# tiprel_1mes  月初客户关系类型 A (active), I (inactive), P (former customer 前客户),R (Potential 潜在客户)
# indresi 居住指数S (Yes) 银行和居住国相同 N (No) 不同国家
# indext  外国人指数, S 本国出生, N 非本国
# conyuemp 配偶指数, S 客户是员工配偶
# canal_entrada
# indfall
# tipodom
# cod_prov 省份编码
# nomprov 省份名称
# ind_actividad_cliente
# renta 家庭总收入
# segmento segmentation: 01 - VIP, 02 - Individuals 03 - college graduated

In [12]:
# cal_loss(df)

In [13]:
# 删除27734条空行, 所有全空
df.drop(df[df['ind_empleado'].isnull()].index, inplace=True)
# 删除全other的空行, 所有全空
df.drop(df[df['pais_residencia'] == 'others'].index, inplace=True)

In [14]:
# 缺失值很少的删除行
df.drop(df[df['sexo'].isnull()].index, inplace=True)
df.drop(df[df['tipodom'].isnull()].index, inplace=True)

In [15]:
# 缺失值很多的删除列
df.drop('ult_fec_cli_1t', axis=1, inplace=True)
df.drop('conyuemp', axis=1, inplace=True)

In [16]:
# 省份名称直接删除, 省份编码缺失值用-1填充
df.drop('nomprov', axis=1, inplace=True)
df.cod_prov.fillna(-1, inplace=True)

In [17]:
# df.loc[(df.ind_nomina_ult1.isna()) | (df.ind_nom_pens_ult1.isna()), y_name_list]

In [18]:
# df.loc[(df.ind_nomina_ult1.isna()) & (df.ind_nom_pens_ult1.isna()), y_name_list].sum(axis=1)

In [19]:
# (df.loc[:, y_name_list].sum(axis=1)==0).sum()

In [20]:
# df[df['indrel_1mes'].isna()].iloc[:, :df.shape[1] - 24]

In [21]:
# canal_entrada 用户加入渠道非常多162种 而且比较平衡, 还是用unknown填充, 考虑KNN填充
df.canal_entrada.value_counts()
df.canal_entrada.fillna('unknown', inplace=True)

KHE    40240
KAT    33027
KFC    30935
KHQ     5992
KFA     4115
       ...  
KBN        1
KCX        1
KBX        1
KBP        1
KFB        1
Name: canal_entrada, Length: 147, dtype: int64

In [22]:
df.renta

2428958     172252.62
10065819          NaN
518173       54872.58
1673167      29403.78
5226724     268779.81
              ...    
1067876     285697.05
12144       134160.78
8704871     235476.72
2064636     324054.99
943302            NaN
Name: renta, Length: 136211, dtype: float64

In [23]:
# renta 数值型填充
# sns.scatterplot(df.renta, df.renta.value_counts())
# df.renta.plot.scatter(x='renta', y=df.renta.value_counts())

In [24]:
# TODO
df.renta.fillna(df.renta.mean(), inplace=True)

In [25]:
df.indrel_1mes.fillna('unknown', inplace=True)
df.tiprel_1mes.fillna('unknown', inplace=True)

In [26]:
# 23. segmento : 01 - VIP, 02 - 个人 03 - 大学毕业
df.segmento.fillna('00', inplace=True)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136211 entries, 2428958 to 943302
Data columns (total 45 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   fecha_dato             136211 non-null  object 
 1   ncodpers               136211 non-null  int64  
 2   ind_empleado           136211 non-null  object 
 3   pais_residencia        136211 non-null  object 
 4   sexo                   136211 non-null  object 
 5   age                    136211 non-null  object 
 6   fecha_alta             136211 non-null  object 
 7   ind_nuevo              136211 non-null  object 
 8   antiguedad             136211 non-null  object 
 9   indrel                 136211 non-null  float64
 10  indrel_1mes            136211 non-null  object 
 11  tiprel_1mes            136211 non-null  object 
 12  indresi                136211 non-null  object 
 13  indext                 136211 non-null  object 
 14  canal_entrada          136211 

In [28]:
cal_loss(df)

Series([], dtype: float64)


# 编码

In [29]:
# 0. fecha_dato 日期
# df.fecha_dato.value_counts(dropna=False).plot(kind='bar')
# 1. ncodpers 客户代码, 弃用
# df.ncodpers.value_counts(dropna=False)

In [30]:
# 日期和客户代码删除
df.drop(['fecha_dato', 'ncodpers'], axis=1, inplace=True)

In [31]:
def label_enc(data):
    label_enc = preprocessing.LabelEncoder()  #获取一个LabelEncoder
    # label_enc = preprocessing.OrdinalEncoder()  #获取一个LabelEncoder
    label_data = label_enc.fit_transform(data)
    return np.array(label_data).reshape(-1, 1)


# def concat_encoder(df):
#     data = pd.concat([data, pd.DataFrame(arrays, columns=names)],axis=1)
#     data = data.drop(['hour'],axis=1)
#     return df

# 对单列编码
def oh_enc(data):
    label_data = label_enc(data)
    # 2. 再用onehot
    oh_enc = preprocessing.OneHotEncoder()
    name_list = []
    for i in range(label_data.classes_.shape[0]):
        name = f'{data.name}_{i}'
        name_list.append(name)
    # 给fit_transform传递的X一定要是 2D的，即（samples，features），否则会报错`
    onehot_data = oh_enc.fit_transform(label_data)
    # 转成numpy array格式
    onehot_data = onehot_data.toarray()
    enc_df = pd.DataFrame(onehot_data, columns=name_list)
    return enc_df

In [32]:

# 2. 5 ind_empleado 员工指标,
# 树模型直接不处理
# A active 活跃, B ex employed 前雇员, F filial 子女, N not employee 非雇员, P pasive 被动, 目前没有寻找新工作, 但是对新工作持开放态度

# df.ind_empleado.value_counts(dropna=False)
# enc_array = label_enc(df.ind_empleado)
def get_ordinal_name(data) -> list:
    return [f'{data.name}_enc']


def get_onehot_name(data) -> list:
    length = data.unique().shape[0]
    l = []
    for i in range(length):
        l.append(f'{data.name}_oh_{i}')
    return l


def new_ordinal_df(data: pd.Series, array):
    name: list = get_ordinal_name(data)
    return pd.DataFrame(array, index=data.index, columns=name)


# TODO
def new_onehot_df(data: pd.Series, array):
    name: list = get_ordinal_name(data)
    return pd.DataFrame(array, index=data.index, columns=name)


# TODO
def show_value_counts(df, od_set: set, oh_set: set):
    df[get_ordinal_name(df.ind_empleado)].value_counts(dropna=False)


od_set = set()
oh_set = set()

In [33]:
l = get_onehot_name(df.ind_empleado)
e = get_ordinal_name(df.ind_empleado)
l
e

['ind_empleado_oh_0',
 'ind_empleado_oh_1',
 'ind_empleado_oh_2',
 'ind_empleado_oh_3']

['ind_empleado_enc']

In [34]:
# df.drop(get_ordinal_name(df.ind_empleado), axis=1, inplace=True)

In [35]:
# df[get_ordinal_name(df.ind_empleado)].value_counts()

df.ind_empleado.value_counts(dropna=False)

N    136127
B        37
F        24
A        23
Name: ind_empleado, dtype: int64

In [36]:
# 单列编码
# df = pd.concat([df, new_ordinal_df(df.ind_empleado, label_enc(df.ind_empleado))], axis=1)
# df[get_ordinal_name(df.ind_empleado)].value_counts(dropna=False)

# 用list收集, 统一进行编码处理
od_set.add(df.ind_empleado.name)

In [37]:
# 3. 118 pais_residencia 客户国籍 ES占比巨大
# 发现others国家的人所有数据都是others
# 继续处理
# Nan unknown
# TODO 计算占比
# df.pais_residencia.value_counts(dropna=False)
# df.pais_residencia.unique()

# 用others替换其他
df.loc[(df['pais_residencia'] != 'ES'), 'pais_residencia'] = 'others'

# df[df['pais_residencia']=='ES'].sum()
df.pais_residencia.value_counts(dropna=False)

od_set.add(df.pais_residencia.name)

ES        135522
others       689
Name: pais_residencia, dtype: int64

In [38]:
# 4. sexo
# Nan = unknown
# 无序类别
df.sexo.value_counts(dropna=False)
od_set.add(df.sexo.name)

V    74119
H    62092
Name: sexo, dtype: int64

In [39]:
df.age = df.age.astype('int8')

In [40]:
# 5. age 年龄
# 清洗>100的异常值
# df.age.value_counts(dropna=False)
low_age = df.loc[df.age < 16, 'age']
low_age.mean()

# df['age'].unique()

11.248447204968944

In [41]:
df.fecha_alta.value_counts(dropna=False)

2014-07-28    609
2014-10-03    557
2014-08-04    433
2013-10-14    414
2013-08-03    335
             ... 
1995-10-14      1
2010-01-01      1
2007-04-14      1
2009-06-21      1
2013-01-19      1
Name: fecha_alta, Length: 6285, dtype: int64

In [42]:
pd.DatetimeIndex(df["fecha_alta"]).month.value_counts()

10    22255
9     16022
11    15439
7     14698
8     13674
12    10074
1      8574
3      7663
2      7579
4      7066
5      6705
6      6462
Name: fecha_alta, dtype: int64

In [43]:
temp = df.fecha_alta.astype('datetime64[M]')
temp.value_counts()

2013-10-01    3793
2014-10-01    3336
2012-10-01    3121
2014-09-01    2883
2014-07-01    2644
              ... 
1995-08-01      37
1995-03-01      24
1995-04-01      18
1995-01-01      14
1995-02-01       8
Name: fecha_alta, Length: 257, dtype: int64

In [44]:
df.antiguedad.value_counts(dropna=False)
od_set.add(df.antiguedad.name)

     12    2380
     21    2133
     10    2049
     23    1774
     33    1733
           ... 
    252       8
    251       7
    253       4
    255       3
    254       2
Name: antiguedad, Length: 256, dtype: int64

In [45]:
df.fecha_alta.info()

<class 'pandas.core.series.Series'>
Int64Index: 136211 entries, 2428958 to 943302
Series name: fecha_alta
Non-Null Count   Dtype 
--------------   ----- 
136211 non-null  object
dtypes: object(1)
memory usage: 2.1+ MB


In [46]:
x = pd.to_datetime(df.fecha_alta)
x.info()

<class 'pandas.core.series.Series'>
Int64Index: 136211 entries, 2428958 to 943302
Series name: fecha_alta
Non-Null Count   Dtype         
--------------   -----         
136211 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 2.1 MB


In [47]:
# 6.fecha_alta 客户首次签订合同的日期, 可以用距今差值表示, 总结成月份, 然后判断和 antiguedad客户资历的关系
df.antiguedad.value_counts(dropna=False)
df.antiguedad.info()

# TODO 暂时先删除
df.drop('fecha_alta', inplace=True, axis=1)
# df.fecha_alta.isnull().sum()
# months_active = df.loc[df["ind_nuevo"].isnull(), :].groupby("ncodpers", sort=False).size()
# months_active.max()

     12    2380
     21    2133
     10    2049
     23    1774
     33    1733
           ... 
    252       8
    251       7
    253       4
    255       3
    254       2
Name: antiguedad, Length: 256, dtype: int64

<class 'pandas.core.series.Series'>
Int64Index: 136211 entries, 2428958 to 943302
Series name: antiguedad
Non-Null Count   Dtype 
--------------   ----- 
136211 non-null  object
dtypes: object(1)
memory usage: 2.1+ MB


In [48]:

# 7.ind_nuevo 新客户指数 2. 如果是6个月内注册的客户, 则为1
df.ind_nuevo = df.ind_nuevo.astype('int8')
df.ind_nuevo.value_counts(dropna=False)

od_set.add(df.ind_nuevo.name)

0    127962
1      8249
Name: ind_nuevo, dtype: int64

In [49]:
# 8. antiguedad 客户资历, 单位是月
df.antiguedad.value_counts(dropna=False)
od_set.add(df.antiguedad.name)

     12    2380
     21    2133
     10    2049
     23    1774
     33    1733
           ... 
    252       8
    251       7
    253       4
    255       3
    254       2
Name: antiguedad, Length: 256, dtype: int64

In [50]:
# 6,7,8 缺失值数量相等已经删除

In [51]:
# 9.indrel  2:  1表示primary, 99表示当月primary,但不是月末?
# Nan =unknown
# one-hot label编码?
df.indrel = df.indrel.astype('int8')
df.indrel.value_counts(dropna=False)
od_set.add(df.indrel.name)

1     135981
99       230
Name: indrel, dtype: int64

In [52]:
# 10. ult_fec_cli_1t 作为主要客户的日期
# 缺失值太多, 丢弃
# df.ult_fec_cli_1t.value_counts(dropna=False)
# df.ult_fec_cli_1t.isnull().sum()

In [53]:
# df.replace('unknown', -2, inplace=True)
# df.replace('P', -3, inplace=True)
# 必须先修改类型, 不然replace匹配不上
# df.indrel_1mes = df.indrel_1mes.astype('float')
df.indrel_1mes.replace({'1.0': '1', '2.0': '2', '3.0': '3', '4.0': '4'}, inplace=True)  # 不同key用不同value替换
df.indrel_1mes.value_counts(dropna=False)
od_set.add(df.indrel_1mes.name)

1          134885
unknown      1250
3              51
2              16
P               5
4               4
Name: indrel_1mes, dtype: int64

In [54]:
# 12.tiprel_1mes 月初客户关系类型 A (active), I (inactive), P (former customer 前客户),R (Potential 潜在客户)
# Nan = unknown
# label 编码
df.tiprel_1mes.value_counts(dropna=False)
od_set.add(df.tiprel_1mes.name)

I          72995
A          61906
unknown     1250
P             55
R              5
Name: tiprel_1mes, dtype: int64

In [55]:
# 13. indresi 居住指数S (Yes) 银行和居住国相同 N (No) 不同国家
# label编码, S=1, N=0
df.indresi.value_counts(dropna=False)
od_set.add(df.indresi.name)


S    135522
N       689
Name: indresi, dtype: int64

In [56]:
# 14. indext 外国人指数, S 本国出生, N 非本国
# label 编码
df.indext.value_counts(dropna=False)
od_set.add(df.indext.name)


N    129768
S      6443
Name: indext, dtype: int64

In [57]:
# 15. conyuemp 配偶指数, S 客户是员工配偶
# 缺失太多, 删除列
# df.conyuemp.value_counts(dropna=False)

In [58]:
# 16. canal_entrada	客户渠道 (拉新渠道)
# Nan = unknown, 占比小于3%都是other
# 用占比, 占比低的作为others
df.canal_entrada.value_counts(dropna=False)
# df.canal_entrada.isnull().sum()
# print(100 * round(df.canal_entrada.value_counts(dropna=False) / df.shape[0], 8))
od_set.add(df.canal_entrada.name)

KHE    40240
KAT    33027
KFC    30935
KHQ     5992
KFA     4115
       ...  
KBN        1
KCX        1
KBX        1
KBP        1
KFB        1
Name: canal_entrada, Length: 148, dtype: int64

In [59]:
# 17. indfall 已故索引 N/S
df.indfall.value_counts(dropna=False)
od_set.add(df.indfall.name)


N    135834
S       377
Name: indfall, dtype: int64

In [60]:
# 18. tipodom 地址类型. 1, primary address 弃用
# Nan dropna
# 没有区分度, 删除列
# label编码
df.tipodom.value_counts(dropna=False)
df.drop('tipodom', axis=1, inplace=True)

1.0    136211
Name: tipodom, dtype: int64

In [61]:
# 19.cod_prov 省份编码
# 转成int
# Nan = 0?
# number
df.cod_prov = df.cod_prov.astype('int8')
df.cod_prov.value_counts(dropna=False)
od_set.add(df.cod_prov.name)


 28    44309
 8     12648
 46     6820
 41     6030
 15     4304
 30     3926
 29     3643
 50     3421
 3      3221
 11     3032
 36     2790
 33     2678
 35     2392
 47     2345
 6      1995
 48     1858
 45     1815
 18     1779
 37     1653
 39     1571
 14     1396
 7      1297
 10     1274
 21     1204
 13     1172
 2      1140
 43     1024
 12     1020
 9       939
 17      908
 26      898
 31      885
 27      834
 24      830
 25      797
 32      757
-1       690
 20      683
 38      656
 19      649
 23      638
 4       590
 16      550
 49      491
 34      449
 40      427
 22      417
 5       392
 1       383
 44      230
 42      208
 52       88
 51       65
Name: cod_prov, dtype: int64

In [62]:
# 20. nomprov 省份名称 删除
# df.nomprov.value_counts(dropna=False)

In [63]:
# 21. ind_actividad_cliente	活跃指数 1, active customer; 0, inactive customer)
df.ind_actividad_cliente = df.ind_actividad_cliente.astype('int8')
df.ind_actividad_cliente.value_counts(dropna=False)
od_set.add(df.ind_actividad_cliente.name)


0    73691
1    62520
Name: ind_actividad_cliente, dtype: int64

In [64]:
df.renta.max()

24253236.93

In [65]:
df.renta.min()

5163.75

In [66]:
df.renta = round(df.renta)
df.renta = df.renta.astype('int32')

In [67]:
# 23. segmento : 01 - VIP, 02 - 个人 03 - 大学毕业
# Nan = unknown
# ordinal 编码
df.segmento.value_counts()
od_set.add(df.segmento.name)


02 - PARTICULARES     79783
03 - UNIVERSITARIO    49268
01 - TOP               5509
00                     1651
Name: segmento, dtype: int64

In [68]:
# 未编码的列
set(df.columns) - od_set - oh_set - set(y_name_list)

{'age', 'renta'}

In [69]:
# 编码流程
len(od_set)


15

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136211 entries, 2428958 to 943302
Data columns (total 41 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   ind_empleado           136211 non-null  object
 1   pais_residencia        136211 non-null  object
 2   sexo                   136211 non-null  object
 3   age                    136211 non-null  int8  
 4   ind_nuevo              136211 non-null  int8  
 5   antiguedad             136211 non-null  object
 6   indrel                 136211 non-null  int8  
 7   indrel_1mes            136211 non-null  object
 8   tiprel_1mes            136211 non-null  object
 9   indresi                136211 non-null  object
 10  indext                 136211 non-null  object
 11  canal_entrada          136211 non-null  object
 12  indfall                136211 non-null  object
 13  cod_prov               136211 non-null  int8  
 14  ind_actividad_cliente  136211 non-null  int8  

In [71]:
finished = []
for name in od_set:
    print(name)
    df = pd.concat([df, new_ordinal_df(df[name], label_enc(df[name]))], axis=1)
    finished.append(name)
df.drop(finished, inplace=True, axis=1)


indfall
indrel
segmento
indresi
indrel_1mes
canal_entrada
ind_nuevo
pais_residencia
tiprel_1mes
antiguedad
sexo
ind_empleado
indext
ind_actividad_cliente
cod_prov


In [72]:
X = df.drop(y_name_list, axis=1)
X

,age,renta,indfall_enc,indrel_enc,segmento_enc,indresi_enc,indrel_1mes_enc,canal_entrada_enc,ind_nuevo_enc,pais_residencia_enc,tiprel_1mes_enc,antiguedad_enc,sexo_enc,ind_empleado_enc,indext_enc,ind_actividad_cliente_enc,cod_prov_enc
2428958,26,172253,0,0,3,1,0,137,0,0,0,46,1,3,0,0,15
10065819,40,133054,0,0,2,1,0,112,0,0,1,23,0,3,0,0,28
518173,44,54873,0,0,2,1,0,112,0,0,0,65,1,3,0,1,35
1673167,43,29404,0,0,2,1,0,22,0,0,1,242,1,3,0,0,28
5226724,23,268780,0,0,3,1,0,137,0,0,1,25,1,3,0,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1067876,24,285697,0,0,3,1,0,137,0,0,1,12,1,3,0,0,15
12144,23,134161,0,0,3,1,0,137,0,0,1,34,0,3,0,0,36
8704871,51,235477,0,0,2,1,0,112,0,0,1,115,0,3,0,0,28
2064636,58,324055,0,0,2,1,0,22,0,0,1,175,0,3,0,0,41


In [73]:
y = df.loc[:, y_name_list]
y

,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
2428958,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
10065819,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
518173,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1673167,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5226724,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1067876,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12144,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8704871,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2064636,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# 模型

In [74]:


Xtrain, Xvalidation, Ytrain, Yvalidation = train_test_split(X, y, test_size=0.3)

In [75]:
Xtrain

,age,renta,indfall_enc,indrel_enc,segmento_enc,indresi_enc,indrel_1mes_enc,canal_entrada_enc,ind_nuevo_enc,pais_residencia_enc,tiprel_1mes_enc,antiguedad_enc,sexo_enc,ind_empleado_enc,indext_enc,ind_actividad_cliente_enc,cod_prov_enc
7425152,50,133054,0,0,2,1,0,22,0,0,0,114,0,3,0,1,1
1365124,67,78992,0,0,2,1,0,22,0,0,0,73,0,3,0,1,28
12652831,24,86845,0,0,3,1,0,137,0,0,1,29,1,3,0,0,33
8641428,48,901080,0,0,2,1,0,22,0,0,1,191,0,3,0,0,28
3488603,46,138139,0,0,2,1,0,112,0,0,1,172,1,3,0,0,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6419366,22,55002,0,0,3,1,0,137,0,0,0,26,1,3,0,1,2
10869495,21,58047,0,0,3,1,0,145,0,0,0,6,0,3,0,1,27
1586439,53,105378,0,0,2,1,0,22,0,0,1,21,1,3,0,1,45
2622409,37,176931,0,0,2,1,0,137,0,0,1,47,0,3,0,1,2


In [76]:
Ytrain

,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
7425152,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1365124,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
12652831,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8641428,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3488603,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6419366,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10869495,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1586439,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2622409,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [77]:
rfc = RandomForestClassifier(random_state=0, n_jobs=18)
rfc = rfc.fit(Xtrain, Ytrain)
y_pred = rfc.predict(Xvalidation)


In [78]:
# score_r = rfc.score(Xvalidation, Yvalidation)

In [79]:
Yvalidation

,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
8720895,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6901261,0,0,0,0,1,0,0,1,0,0,...,0,1,0,0,0,1,0,1,1,1
6730488,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2031368,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
7611018,0,0,1,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9137125,0,0,0,0,1,0,0,0,1,0,...,1,0,0,1,1,0,0,1,1,1
7039864,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4433010,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11808453,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [80]:
y_pred

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]], dtype=uint8)

In [103]:
from sklearn.metrics import plot_roc_curve

from sklearn.metrics import ConfusionMatrixDisplay
import sklearn.metrics as metrics


def print_evaluation(y_true, y_pred, target_names=None, average='micro'):
    report = classification_report(y_true, y_pred)

    # ConfusionMatrixDisplay.from_predictions(
    #     y_true, predicted_labels, labels=lp_model.classes_
    # )
    acc = metrics.accuracy_score(y_true, y_pred)

    # auc_score = roc_auc_score(y_true, y_pred)  #验证集上的auc值
    # svc_disp = plot_roc_curve(rfc, Xvalidation, y_pred)
    # plt.show()

    # confusion_matrix = metrics.multilabel_confusion_matrix(y_true, y_pred)
    #
    # precision = metrics.precision_score(y_true, y_pred, average=average)
    # metrics.recall_score(y_true, y_pred, average=average)
    # metrics.f1_score(y_true, y_pred, average=average)
    #
    # 统一precision+recall+f1
    prf = metrics.precision_recall_fscore_support(y_true, y_pred, average=average)
    # hamming = metrics.hamming_loss(y_true, y_pred)
    # kappa = metrics.cohen_kappa_score(y_true, y_pred)

    print()
    print(f"acc \n{acc}")
    print(f"report \n{report}")
    print(f'precision_recall_fscore_support\n{prf}\n')
    # print(f'\n{hamming}\n')
    # print(f'\n{kappa}\n')
    print(report)

print_evaluation(Yvalidation, y_pred)


/home/lucas/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lucas/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lucas/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lucas/ana

ValueError: multilabel-indicator is not supported

In [ ]:
set(Yvalidation) - set(y_pred)

In [83]:
auc_score = metrics.roc_auc_score(Yvalidation, y_pred)  #验证集上的auc值


ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [ ]:
# GBDT
GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)